# Building an Energy Model

In this notebook, we will build a model of the Austrian energy system from scratch. The process will involve defining our model's time horizon and spatial extent, and then populating the model with data associated with model parameters. Once we have a basline model, we will then move on to investigating policy scenarios.

We will be populating different kinds of parameters including

### Economic Parameters

- `interestrate`
- `demand`

### Technology Parameters

#### Engineering Parameters

- `input`
- `output`
- `technical_lifetime`
- `capacity_factor`


#### Technoeconomic Parameters

- `inv_cost`
- `fix_cost`
- `var_cost`

### Dynamic Behavior Parameters

- `bound_activity_up`
- `bound_activity_lo`
- `bound_new_capacity_up`
- `initial_activity_up`
- `growth_activity_up`

### Emissions

- `emission_factor`

A full list of parameters can be found on the (internal) [MESSAGEix documentation website](http://ienecat.iiasa.ac.at:8787/message_ix_doc). This website can be built in the future from the repository that you are using. 

## The Final Product

At the completion of this exercise, we will have developed an energy model is comprised of the below Reference Energy System (RES):

![title](austria.png)


## Set options

In [ ]:
# Enter name of the input file that should be read
fname = 'austria_input.xlsx'
# Choose whether or not data entered into the datastrucuture should be displayed (True or False)
verbose = False
# Choose whether or not data input errors are shown (True or False)
disp_error = False

## Setup

In [ ]:
# load required packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import message_ix
from message_ix import xlsx_importer
from ixmp import Platform
mp = Platform(dbtype='HSQLDB')

im = xlsx_importer.init_model(mp, fname, verbose, disp_error)

## Read in input data

In [ ]:
meta, tecs, dems, resources, mpa_data = im.read_input()

## Create scenario

In [ ]:
scenario, model_nm, scen_nm = im.create_scen()

## Setup scenario metadata

In [ ]:
horizon, vintage_years, firstyear = im.add_metadata()

# Process input data

## Import class add_par from xlsx_core

In [ ]:
ap = xlsx_importer.add_par(scenario, horizon, vintage_years, firstyear, disp_error)

## Process demand data

In [ ]:
im.demand_input_data(ap)

## Process technology data

In [ ]:
im.technology_parameters(ap)

# Custom scenario functions

In [ ]:
scenario.add_cat('emission', 'GHGs', 'CO2')

# Solve

In [ ]:
comment = 'initial commit for Austria model'
scenario.commit(comment)
scenario.set_as_default()

In [ ]:
scenario.solve(model='MESSAGE')

# Postprocessing

In [ ]:
scenario.var("OBJ")

In [ ]:
from tools import Plots
p = Plots(scenario, 'Austria')

In [ ]:
plants = xlsx_importer.apply_filters(tecs, filters={'Commodity': 'electricity', 'Level': ['secondary','final'], 'Parameter': 'output'})
plants = plants['Technology'].dropna().unique().tolist()
plants = [t for t in plants if t not in ['import','electricity_grid']]
p.plot_new_capacity(baseyear=True, subset=plants)

In [ ]:
lights = xlsx_importer.apply_filters(tecs, filters={'Commodity': 'light', 'Level': 'useful', 'Parameter': 'output'})
lights = lights['Technology'].dropna().unique().tolist()
p.plot_new_capacity(baseyear=True, subset=lights)

In [ ]:
p.plot_capacity(baseyear=True, subset=plants)

In [ ]:
p.plot_capacity(baseyear=True, subset=lights)

In [ ]:
elec_demand = scenario.par("demand", filters={'commodity': ['other_electricity']})
light_demand = scenario.par("demand", filters={'commodity': ['light']})
p.plot_demand(light_demand, elec_demand)

In [ ]:
p.plot_activity(baseyear=True, subset=plants)

In [ ]:
p.plot_activity(baseyear=True, subset=lights)

In [ ]:
p.plot_prices(baseyear=False, subset=['light', 'other_electricity'])

In [ ]:
mp.close_db()